In [1]:
import pandas as pd
import pandasql as ps

In [2]:
ifood = pd.read_csv("ifood-restaurants-february-2021.csv")

# Visualizando as primeiras linhas
ps.sqldf("SELECT * FROM ifood where rating >= 4.9 limit 3")

,availableForScheduling,avatar,category,delivery_fee,delivery_time,distance,ibge,minimumOrderValue,name,paymentCodes,price_range,rating,tags,url
0,0,https://static-images.ifood.com.br/image/uploa...,Pizza,0.00,25,1.06,3550308,28.0,Pizzaria Sabor Sem Igual,DNR $$ MPAY $$ MOVPAY_MC $$ RAM $$ MC $$ GPY_E...,CHEAPEST,5.0,ADDRESS_PREFORM_TYPE $$ CPGN_USER_DISCOUNT_BR_...,https://www.ifood.com.br/delivery/sao-paulo-sp...
1,0,https://static-images.ifood.com.br/image/uploa...,Pizza,0.00,30,3.70,3550308,0.0,Pizzeria Aurichi,DNR $$ MPAY $$ MOVPAY_MC $$ MC $$ GPY_ELO $$ R...,CHEAPEST,5.0,ADDRESS_PREFORM_TYPE $$ CPGN_USER_DISCOUNT_BR_...,https://www.ifood.com.br/delivery/sao-paulo-sp...
2,0,https://static-images.ifood.com.br/image/uploa...,Pastel,12.99,37,6.85,3550308,20.0,A Garagem Pastéis e Bebidas,DNR $$ MPAY $$ MOVPAY_MC $$ MC $$ GPY_ELO $$ E...,CHEAPEST,5.0,ADDRESS_PREFORM_TYPE $$ GUIDED_HELP_TYPE $$ NO...,https://www.ifood.com.br/delivery/sao-paulo-sp...


In [15]:
# Tipo de dados das colunas
for i in ifood.columns:
    display(ps.sqldf(f"SELECT DISTINCT typeof({i}) FROM ifood"))

,typeof(availableForScheduling)
0,integer


,typeof(avatar)
0,text
1,null


,typeof(category)
0,text


,typeof(delivery_fee)
0,real


,typeof(delivery_time)
0,integer


...

,typeof(paymentCodes)
0,text
1,null


,typeof(price_range)
0,text


,typeof(rating)
0,real


,typeof(tags)
0,text


,typeof(url)
0,text


In [18]:
# Contagem de nulos/não nulos
ps.sqldf( """
                    SELECT 
                        SUM(CASE WHEN paymentCodes IS NULL THEN 1 ELSE 0 END) as countar_nulls
                        , SUM(CASE WHEN paymentCodes IS NULL THEN 0 ELSE 1 END) as countar__n_nulls
                    FROM ifood
               ;"""

)

,countar_nulls,countar__n_nulls
0,2,406397


In [21]:
# Valores distintos em cada coluna
for i in ifood.columns:
    display(ps.sqldf(f"""
                    SELECT 
                        COUNT(DISTINCT {i})
                    FROM ifood
               """))

,COUNT(DISTINCT availableForScheduling)
0,2


,COUNT(DISTINCT avatar)
0,406116


,COUNT(DISTINCT category)
0,61


,COUNT(DISTINCT delivery_fee)
0,406


,COUNT(DISTINCT delivery_time)
0,190


...

,COUNT(DISTINCT paymentCodes)
0,18376


,COUNT(DISTINCT price_range)
0,5


,COUNT(DISTINCT rating)
0,22471


,COUNT(DISTINCT tags)
0,118660


,COUNT(DISTINCT url)
0,406399


In [33]:
# Pegunta 1: Qual categoria mais aparece na base?
ps.sqldf(
""" SELECT 
        category as Categoria,
        COUNT(category) AS Contagem
    FROM ifood
    GROUP BY category
    ORDER BY contagem DESC
    LIMIT 1
"""
)

,Categoria,Contagem
0,Lanches,98946


In [38]:
# Pergunta 2: Qual a maior taxa de entrega da base?
ps.sqldf(
"""
    SELECT 
        MAX(delivery_fee) as 'Maior taxa de entrega'
    FROM 
        ifood

"""

)

,Maior taxa de entrega
0,35.0


In [35]:
# Pergunta 3: Qual o tempo médio de entrega?
ps.sqldf(
"""
    SELECT 
        AVG(delivery_time) as 'Tempo médio de entrega'
    FROM 
        ifood
"""
)

,Tempo médio de entrega
0,47.432747


In [39]:
# Pergunta 4: Qual a média de taxa de entrega?
ps.sqldf(
"""
    SELECT 
        AVG(delivery_fee) as 'Taxa média de entrega'
    FROM 
        ifood
"""
)

,Taxa média de entrega
0,6.804577


In [40]:
# Pergunta 5: Qual o maior valor mínimo de pedido?
ps.sqldf(
"""
    SELECT 
        MAX(minimumOrderValue) as 'Maior Pedido Mínimo'
    FROM 
        ifood
"""
)
# Aqui há uma inconsistencia e deve ser verificado se há problema com a base, a captura dos dados ou com o cadastro do restaurante

,Maior Pedido Mínimo
0,99999999.99


In [57]:
# Pergunta 6: Quantos restaurantes avaliados com nota 5 existem para cada categoria e qual o total geral?
ps.sqldf(
"""
    SELECT 
        DISTINCT category AS category,
        COUNT(rating) OVER(PARTITION BY category) AS 'Restaurantes Nota 5 por categoria',
        COUNT(rating) OVER() AS 'Total de Restautantes nota 5'
    FROM 
        ifood
    WHERE
        rating = 5.0
         
"""
)


,category,Restaurantes Nota 5 por categoria,Total de Restautantes nota 5
0,Africana,6,49296
1,Alemã,15,49296
2,Argentina,30,49296
3,Asiática,19,49296
4,Açaí,3359,49296
5,Baiana,21,49296
6,Bebidas,3439,49296
7,Brasileira,6321,49296
8,Cafeteria,507,49296
9,Carnes,864,49296


In [59]:
# Pergunta 7: Qual a média de notas das avaliações dos restaurantes na base (Total e por categoria)?
ps.sqldf(
"""
    SELECT 
        DISTINCT category AS category,
        AVG(rating) OVER(PARTITION BY category) AS 'Restaurantes Nota 5 por categoria',
        AVG(rating) OVER() AS 'Total de Restautantes nota 5'
    FROM 
        ifood      
"""
)

,category,Restaurantes Nota 5 por categoria,Total de Restautantes nota 5
0,Africana,1.628755,2.522241
1,Alemã,2.698104,2.522241
2,Argentina,1.868734,2.522241
3,Asiática,3.277633,2.522241
4,Açaí,2.932867,2.522241
...,...,...,...
56,Vegana,3.615712,2.522241
57,Vegetariana,2.849850,2.522241
58,Xis,2.758929,2.522241
59,Yakisoba,2.751846,2.522241


In [7]:
# Pergunta 8: Os restaurantes mais bem avaliados são os mais caros ?
ps.sqldf(
"""
SELECT 
    DISTINCT A.price_range AS 'Classificação de Preços',
    AVG(A.rating) AS 'Média das avaliações',
    B.moda
    
       
FROM ifood AS A
INNER JOIN (SELECT 
            rating AS moda,
            price_range,
            COUNT(rating) AS frequencia
        FROM ifood
        GROUP BY price_range
        ORDER BY frequencia  ) AS B ON A.price_range = B.price_range

GROUP BY "Classificação de Preços"
ORDER BY "Classificação de Preços" DESC
"""
)


,Classificação de Preços,Média das avaliações,moda
0,MOST_EXPENSIVE,4.058625,0.0
1,MODERATE,3.671159,0.0
2,EXPENSIVE,3.815964,0.0
3,CHEAPEST,2.118592,0.0
4,CHEAP,3.302978,0.0


In [ ]:
# Pergunta 9: Qual categoria tem os restaurantes mais bem avaliados?

ps.sqldf(
""" 
    SELECT 

"""


)

In [69]:
ps.sqldf("SELECT * FROM ifood where rating >= 4.9 limit 3")

,availableForScheduling,avatar,category,delivery_fee,delivery_time,distance,ibge,minimumOrderValue,name,paymentCodes,price_range,rating,tags,url
0,0,https://static-images.ifood.com.br/image/uploa...,Pizza,0.00,25,1.06,3550308,28.0,Pizzaria Sabor Sem Igual,DNR $$ MPAY $$ MOVPAY_MC $$ RAM $$ MC $$ GPY_E...,CHEAPEST,5.0,ADDRESS_PREFORM_TYPE $$ CPGN_USER_DISCOUNT_BR_...,https://www.ifood.com.br/delivery/sao-paulo-sp...
1,0,https://static-images.ifood.com.br/image/uploa...,Pizza,0.00,30,3.70,3550308,0.0,Pizzeria Aurichi,DNR $$ MPAY $$ MOVPAY_MC $$ MC $$ GPY_ELO $$ R...,CHEAPEST,5.0,ADDRESS_PREFORM_TYPE $$ CPGN_USER_DISCOUNT_BR_...,https://www.ifood.com.br/delivery/sao-paulo-sp...
2,0,https://static-images.ifood.com.br/image/uploa...,Pastel,12.99,37,6.85,3550308,20.0,A Garagem Pastéis e Bebidas,DNR $$ MPAY $$ MOVPAY_MC $$ MC $$ GPY_ELO $$ E...,CHEAPEST,5.0,ADDRESS_PREFORM_TYPE $$ GUIDED_HELP_TYPE $$ NO...,https://www.ifood.com.br/delivery/sao-paulo-sp...


In [ ]:



# Pergunta 10: Qual o valor médio de entrega por km para os restaurantes que tem essa taxa?
